# Interrupt: 그래프 실행 중단 및 재개

이 노트북에서는 **그래프 실행을 특정 지점에서 중단하고 재개**하는 방법을 배웁니다.

## Interrupt란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Interrupt의 개념                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 실행:                                                       │
│   START → 노드1 → 노드2 → 노드3 → END                             │
│   (중간에 멈출 수 없음)                                            │
│                                                                    │
│   interrupt_before 사용:                                           │
│   START → 노드1 → [중단] → 사용자 승인 → 노드2 → 노드3 → END     │
│                                                                    │
│   사용 사례:                                                       │
│   • 도구 실행 전 사용자 승인                                       │
│   • 결제/삭제 등 위험한 작업 전 확인                               │
│   • Human-in-the-loop 워크플로우                                   │
│   • 디버깅 및 검토                                                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    Interrupt 동작 흐름                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ 첫 번째 실행                                                 │
│   graph.stream(input, config, interrupt_before=["model"])         │
│   → select_tools 실행 → model 전에 중단 → 상태 저장               │
│                                                                    │
│   2️⃣ 사용자 검토/승인                                             │
│   (여기서 상태 확인, 수정 가능)                                    │
│                                                                    │
│   3️⃣ 두 번째 실행 (재개)                                          │
│   graph.stream(None, config)  # None = 저장된 상태에서 재개       │
│   → model 실행 → tools 실행 → ... → 완료                          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph duckduckgo-search langchain-community

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. 에이전트 그래프 준비

In [ ]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_ollama import ChatOllama, OllamaEmbeddings

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]

embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)

tools_retriever = InMemoryVectorStore.from_documents(
    [Document(tool.description, metadata={'name': tool.name}) for tool in tools],
    embeddings,
).as_retriever()


class State(TypedDict):
    messages: Annotated[list, add_messages]
    selected_tools: list[str]


def model_node(state: State) -> State:
    selected_tools = [tool for tool in tools if tool.name in state['selected_tools']]
    res = model.bind_tools(selected_tools).invoke(state['messages'])
    return {'messages': res}


def select_tools(state: State) -> State:
    query = state['messages'][-1].content
    tool_docs = tools_retriever.invoke(query)
    return {'selected_tools': [doc.metadata['name'] for doc in tool_docs]}


builder = StateGraph(State)
builder.add_node('select_tools', select_tools)
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))
builder.add_edge(START, 'select_tools')
builder.add_edge('select_tools', 'model')
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

# MemorySaver 필수 (상태 저장용)
graph = builder.compile(checkpointer=MemorySaver())

print("✅ 에이전트 그래프 준비 완료")

# 3. interrupt_before 사용

특정 노드 실행 **전에** 중단합니다.

In [ ]:
input_data = {
    'messages': [
        HumanMessage(
            '미국 제30대 대통령의 사망 당시 나이는 몇 살이었나요?'
        )
    ]
}

config = {"configurable": {"thread_id": "1"}}

print("=== 첫 번째 실행 (model 전에 중단) ===")
print(f"질문: {input_data['messages'][0].content}\n")

# interrupt_before=["model"]: model 노드 실행 전에 중단
output = graph.stream(input_data, config, interrupt_before=["model"])

for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        print(f"  출력: {node_output}")
    print()

print("🛑 model 노드 전에 중단됨")

# 4. 중단된 상태 확인

In [ ]:
# 현재 저장된 상태 확인
state = graph.get_state(config)

print("=== 중단된 상태 ===")
print(f"메시지: {state.values.get('messages', [])}")
print(f"선택된 도구: {state.values.get('selected_tools', [])}")
print(f"\n다음 실행할 노드: {state.next}")

# 5. 재개 (Resume)

In [ ]:
print("=== 두 번째 실행 (재개) ===")
print("사용자가 승인했다고 가정\n")

# None을 전달하면 저장된 상태에서 재개
output = graph.stream(None, config)

for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        if 'messages' in node_output:
            last_msg = node_output['messages'][-1] if isinstance(node_output['messages'], list) else node_output['messages']
            if hasattr(last_msg, 'content') and last_msg.content:
                print(f"  응답: {str(last_msg.content)[:200]}...")
    print()

print("✅ 완료")

# 6. 새 입력으로 재개

In [ ]:
# 새로운 스레드로 시작
config2 = {"configurable": {"thread_id": "2"}}

print("=== 첫 번째 실행 (중단) ===")
output = graph.stream(input_data, config2, interrupt_before=["model"])
for chunk in output:
    pass  # 출력 생략
print("🛑 중단됨\n")

# 새 입력으로 재개 (질문 변경)
new_input = {
    'messages': [
        HumanMessage(
            '영화 보이후드의 촬영 기간은 얼마나 되나요?'
        )
    ]
}

print("=== 두 번째 실행 (새 입력으로 재개) ===")
print(f"새 질문: {new_input['messages'][0].content}\n")

# 새 입력 전달 (기존 상태에 추가됨)
output = graph.stream(new_input, config2)

for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        if 'messages' in node_output:
            last_msg = node_output['messages'][-1] if isinstance(node_output['messages'], list) else node_output['messages']
            if hasattr(last_msg, 'content') and last_msg.content:
                print(f"  응답: {str(last_msg.content)[:200]}...")
    print()

---

## 정리: Interrupt

### 핵심 코드

```python
# 1. checkpointer 필수
graph = builder.compile(checkpointer=MemorySaver())

# 2. interrupt_before로 중단 지점 지정
output = graph.stream(input, config, interrupt_before=["model"])

# 3. 상태 확인
state = graph.get_state(config)
print(state.values)  # 현재 상태
print(state.next)    # 다음 실행할 노드

# 4. 재개 (저장된 상태에서)
output = graph.stream(None, config)

# 5. 새 입력으로 재개
output = graph.stream(new_input, config)
```

### interrupt_before vs interrupt_after

| 옵션 | 중단 시점 | 사용 시나리오 |
|------|----------|---------------|
| **interrupt_before** | 노드 실행 전 | 실행 전 승인 |
| **interrupt_after** | 노드 실행 후 | 결과 검토 후 진행 |

### 사용 사례

| 사례 | 설명 |
|------|------|
| 도구 승인 | 위험한 도구 실행 전 확인 |
| 결제 확인 | 결제 API 호출 전 승인 |
| 데이터 삭제 | 삭제 작업 전 확인 |
| Human-in-the-loop | 사람의 검토 필요 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
embeddings = OpenAIEmbeddings()
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)

# 변경
embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)
```

## 다음 단계

**상태 수정** 및 **히스토리에서 Fork**하는 방법을 배웁니다. (07-11번 노트북)